# Robotic Arm: Pick & Place

## Kinematic Analysis
---

[comment]: <> (IMAGES REFERENCES)

[DH_img]: ./images/DH_parameters2.jpg
[angles123]: ./images/theta123.jpg


### 1 - Run the forward_kinematics demo and evaluate the kr210.urdf.xacro file to perform kinematic analysis of Kuka KR210 robot and derive its DH parameters.

As we saw in Lesson 2 we can simplify the rotation matrix using the Denavit–Hartenberg parameters. We select the rotation in the Z axis of the new axial references. For that we need to assign the values of alpha, a, d and theta adapting the axis defined in the kr210.urdf.xacro file to our new defined model:

The values  can be find beetwen the lines 317 and 363 of the file mentioned above.

![][DH_img]


Following the picuter and the DH table is the following. The selection of the angle Z as the only moving part is a clever simplification for working with rotation matrix and easing the calculation load for the software. In the link number two we nedd to adapt the rest position substracting 90 degrees.


| i | <p> &alpha;_i-1</p> | a_i-1 | d_i | <p> &theta;_i </p>
|:---: | :---: | :---: | :---: | :---:
|1 | 0 |0  | 0.75 | <p> &theta;_1 </p>
|2 | <p>-&pi;/2</p> | 0.35 | 0| <p> &theta;_2  -&pi;/2 </p>
|3 | 0 | 1.25 | 0 | <p> &theta;_3 </p>
|4 | <p>-&pi;/2</p> | -0.054 | 1.5 | <p> &theta;_4 </p>
|5 | <p>-&pi;/2</p> | 0 | 0 | <p> &theta;_5 </p>
|6 | <p>-&pi;/2</p> | 0 | 0 | <p> &theta;_6 </p>
|7 | 0 | 0 | 0.303 | 0


---

### 2 - Using the DH parameter table you derived earlier, create individual transformation matrices about each joint. In addition, also generate a generalized homogeneous transform between base_link and gripper_link using only end-effector(gripper) pose

As we saw in the lessons the generalized homogeneus transformation matrix can be obtained as a multiplication of one of eache transformation matrix between the base and the base link.

First we define the individual transformation matrix for the DH parameters:

``` python
def get_DH_transform_matrix(alpha,a,d,tetha):

    transform = Matrix([ [           cos(tetha),          -sin(tetha),          0,            a],
                        [sin(tetha)*cos(alpha),cos(tetha)*cos(alpha),-sin(alpha),-sin(alpha)*d],
                        [sin(tetha)*sin(alpha),cos(tetha)*sin(alpha), cos(alpha), cos(alpha)*d],
                        [                   0,                     0,          0,            1]])

    return transform
```

Now is easy to generate each one of the individual matrix as follow

``` python
#Generate the DH matrixs
T0_1 = simplify(get_DH_transform_matrix(alpha0,a0,d1,tetha1))
T0_1 = T0_1.subs(s)
T1_2 = simplify(get_DH_transform_matrix(alpha1,a1,d2,tetha2))
T1_2 = T1_2.subs(s)
T2_3 = simplify(get_DH_transform_matrix(alpha2,a2,d3,tetha3))
T2_3 = T2_3.subs(s)
T3_4 = simplify(get_DH_transform_matrix(alpha3,a3,d4,tetha4))
T3_4 = T3_4.subs(s)
T4_5 = simplify(get_DH_transform_matrix(alpha4,a4,d5,tetha5))
T4_5 = T4_5.subs(s)
T5_6 = simplify(get_DH_transform_matrix(alpha5,a5,d6,tetha6))
T5_6 = T5_6.subs(s)
T6_G = simplify(get_DH_transform_matrix(alpha6,a6,d7,tetha7))
T6_G = T6_G.subs(s)

```

And then multiply them to obtain the generalized homogeneous transformation matrix:

``` python
#Multiplication of transformation Matrix
T0_2 = simplify(T0_1*T1_2)
T0_3 = simplify(T0_2*T2_3)
T0_4 = simplify(T0_3*T3_4)
T0_5 = simplify(T0_4*T4_5)
T0_6 = simplify(T0_5*T5_6)
T0_G = simplify(T0_6*T6_G)
```

for the last one we need to adjust the axis as descripbed in the lessons tho the gripper end

``` python
#definition of rotation matrix between x,y and z
def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,        cos(q), -sin(q)],
                  [ 0,        sin(q),  cos(q)]])
    return R_x

def rot_y(q):
    R_y = Matrix([[ cos(q),        0,  sin(q)],
                  [       0,        1,        0],
                  [-sin(q),        0,  cos(q)]])
    return R_y

def rot_z(q):
    R_z = Matrix([[ cos(q), -sin(q),        0],
                  [ sin(q),  cos(q),        0],
                  [ 0,              0,        1]])
    return R_z

#Generalized homogeneous matrix for a rotation in x and y axis
trans = Matrix([[0],[0],[0]])
col = Matrix([[0,0,0,1]])
R_z = (rot_z(pi).row_join(trans)).col_join(col)
R_y = (rot_y(-pi/2).row_join(trans)).col_join(col)

#end efector rotations for correlation
T0_G = T0_G*R_z*R_y
```
---

### 3 - Decouple Inverse Kinematics problem into Inverse Position Kinematics and inverse Orientation Kinematics; doing so derive the equations to calculate all individual joint angles.

The first step for this calculation is find the WC (wrist center) that in case of our model is the point where the joints 4, 5 and 6 can be found. As we know the position and orientation of the EE(end effector) from the pose of the kuka arm, we can go backward tho the joints 4,5,6 as is it explained in the lessons.

``` python
    #Rrpy is the rotation matrix that we calculate with the orientation of the EE
    # R_Z, R_Y and R_X are the indivual rotation matriz around axis Z,Y,X respectively
    Rrpy = R_Z * R_Y * R_X
    # We need to do two aditional rotation for compliance with our model
    Rot_Error = R_Z.subs(y, pi) * R_Y.subs(p, -pi/2.0)
    ROT_EE = Rrpy* Rot_Error
    ROT_EE = ROT_EE.subs({'r': roll, 'y': yaw, 'p': pitch})
    # P_ee is the position of the EE extracted from the pose
    P_ee = Matrix([[px], [py],[pz]])
    #finally we calculate the Wrist position as a translation (d7=0.303) from the EE toward the WC
    WC_00 = P_ee - s[d7] * ROT_EE[:,2]
    WC_00 = WC_00.subs(s)

```

For the calculation of the first 3 angles we use trigonometry and the position of the WC as tools, and image representing the selected variables and the code to calculate the angles can be found below.

The trigonometric analysis offers two different solutions. I have taken the one that is more similar to the kuka arm with the first joint ortogonal from ground.

![][angles123]

``` python
    #calculate q1
    q1 = atan2(WC_00[1],WC_00[0]).subs(s)
    #first need the position of the WC in the 0_2 frame
    WC_z_1 = WC_00[2]-s[d1]
    WC_x_1 = sqrt(WC_00[0]**2 + WC_00[1]**2) - s[a1]
    WC_y_1 = 0
    # aux triangle
    a = sqrt(WC_x_1**2 + WC_z_1**2)
    b = 1.25
    c = sqrt(1.5**2 + 0.054**2)
    # Aux angles
    betha1 = atan2(WC_z_1,WC_x_1)
    betha2 = acos((-c**2 + a**2 + b**2)/(2*a*b))
    betha3 = acos((b**2 + c**2 - a**2)/(2*b*c))
    betha4 = atan2(s[a3],s[d4])
    #q2, q3
    q2 = pi/2 - betha1 - betha2
    q3 = pi/2 - betha3 + betha4
```

The next step is calculate the tranformation matrix from joint 0 to joint 3 and with it calculate the rotation matrix for the end effector R3_6. Following the rotation matrix maths this last one can be calculated with the following code.

``` python
    # we take the rotation matrix from the point 2  --> T0_3 and replace theta 1,2,3 with q1,q2,q3
    R0_3 = T0_3[0:3,0:3]
    R0_3 = R0_3.subs(angles)
    
    R3_6 = R0_3.inv("LU") * ROT_EE
```

The next 3 euler angles can be extrated as we learned in the lesson 2.8, the code is as follow.

``` python
    q4 = atan2(R3_6[2,2], -R3_6[0,2])
    q5 = atan2(sqrt(R3_6[0,2] * R3_6[0,2] + R3_6[2,2] * R3_6[2,2]), R3_6[1,2])
    q6 = atan2(-R3_6[1,1], R3_6[1,0])
    
```

---

## Project Implementation
---

### 1 - Fill in the IK_server.py file with properly commented python code for calculating Inverse Kinematics based on previously performed Kinematic Analysis. Your code must guide the robot to successfully complete 8/10 pick and place cycles.

For the implementation of the project I have add the transformation matrix to a pickle file container (T0_X-matrix.pkl), the steps for that can be follow in the file (forward_kinematics.py)

Additionally the file (IK_calculation.py) a graphical analysis that I used for the visualization of the steps and debugging before the IK_debug.py file was available in the main repository.

The commmented final code can be found in the file (.kuka_arm\scripts\IK_server.py)


